In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

model = ChatOllama(model='llama3.2')

parser = StrOutputParser()

class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

chain = classifier_chain | branch_chain

result=chain.invoke({'feedback': 'This is a good phone'})

chain.get_graph().print_ascii()

print(result)

              +---------+
              |  Input  |
              +---------+
                   |
                   v
         +----------------+
         | PromptTemplate  |
         +----------------+
                   |
                   v
            +-------------+
            | ChatOllama   |
            +-------------+
                   |
                   v
    +----------------------------+
    | PydanticOutputParser (Feedback) |
    +----------------------------+
                   |
                   v
           +-----------------+
           | RunnableBranch   |
           +-----------------+
            /       |        \
           /        |         \
          v         v          v
+----------------+  +----------------+  +------------------------+
| PromptTemplate |  | PromptTemplate |  | RunnableLambda          |
| (positive)     |  | (negative)     |  | (Default: No sentiment) |
+----------------+  +----------------+  +------------------------+
       |                    |                      |
       v                    v                      v
 +-------------+       +-------------+        +-----------------+
 | ChatOllama  |       | ChatOllama  |         |    Static Text   |
 +-------------+       +-------------+        +-----------------+
       |                    |                      
       v                    v                      
+----------------+    +----------------+
| StrOutputParser|    | StrOutputParser|
+----------------+    +----------------+
           \             /
            \           /
             v         v
          +----------------------+
          |   Final Output Merge  |
          +----------------------+


So this is conditional chain where we use runnablebranch for conditions and runnable lembda for default output